In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import sys
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import dca
from dca.dca import DynamicalComponentsAnalysis
from sklearn.model_selection import KFold

In [3]:
sys.path.append('../..')

In [22]:
import pdb

In [39]:
import loaders
import decoders
from loaders import load_shenoy_large
from decoders import lr_decoder
from os import listdir
from os.path import isfile, join
import os
import contextlib
from IPython.utils import io

def lr_decoder_crossval(X, Z, lag, decoding_window):
    kf = KFold(n_splits=4)
    lr_r2_pos= lr_r2_vel = lr_r2_acc = 0
    X = np.squeeze(X)
    pdb.set_trace()
    for train_idx, test_idx in kf.split(X):
        # X_test = [X[idx] for idx in test_idx]
        # X_train = [X[idx] for idx in train_idx]
        # Z_test = [Z[idx] for idx in test_idx]
        # Z_train = [Z[idx] for idx in train_idx]
        X_train = X[train_idx]
        X_test =X[test_idx]
        Z_train = Z[train_idx]
        Z_test = Z[test_idx]
        result = lr_decoder(X_test, X_train, Z_test, Z_train, lag, lag, decoding_window)
        lr_r2_pos += result[0]
        lr_r2_vel += result[1]
        lr_r2_acc += result[2]
    lr_r2_acc /= 4
    lr_r2_pos /= 4
    lr_r2_vel /= 4
    return lr_r2_pos, lr_r2_vel, lr_r2_acc
    
def trial_lag_decodewin(file_name, dat):
    X = dat['spike_rates'] 
    Z = dat['behavior']
    lag_trials = [0, 5, 10, 15, 20]
    decoding_windows = [1, 3, 5]
    max_lr_r2_pos, max_lr_r2_vel, max_lr_r2_acc = -1, -1, -1
    max_pos_lag,max_pos_decoding_window = 0,1
    max_vel_lag,max_vel_decoding_window = 0,1
    max_acc_lag,max_acc_decoding_window = 0,1
    
    
    for lag in lag_trials:
        for decoding_window in decoding_windows:
            lr_r2_pos, lr_r2_vel, lr_r2_acc = lr_decoder_crossval(X, Z, lag, decoding_window)
            if lr_r2_pos > max_lr_r2_pos:
                max_lr_r2_pos = lr_r2_pos
                max_pos_lag,max_pos_decoding_window = lag,decoding_window
            if lr_r2_vel > max_lr_r2_vel:
                max_lr_r2_vel = lr_r2_vel
                max_vel_lag,max_vel_decoding_window = lag,decoding_window
            if lr_r2_acc > max_lr_r2_acc:
                max_lr_r2_acc = lr_r2_acc
                max_acc_lag,max_acc_decoding_window = lag,decoding_window
            #print("Lag: {0}, decoding_window: {1}".format(lag, decoding_window))
            #print("Pos R^2: {0}, Vel R^2: {1}, Acc R^2: {2}".format(lr_r2_pos, lr_r2_vel, lr_r2_acc))
    
    print("Best pos r2: {0}, with lag: {1}, windows: {2}".format(max_lr_r2_pos, max_pos_lag,max_pos_decoding_window))
    print("Best vel r2: {0}, with lag: {1}, windows: {2}".format(max_lr_r2_vel, max_vel_lag,max_vel_decoding_window))
    print("Best acc r2: {0}, with lag: {1}, windows: {2}".format(max_lr_r2_acc, max_acc_lag,max_acc_decoding_window))

In [5]:
path = '/mnt/Secondary/data'

In [6]:
JFiles = [f for f in listdir("%s/000121/sub-JenkinsC" % path) if isfile(join("%s/000121/sub-JenkinsC" %path, f))]
RFiles = [f for f in listdir("%s/000121/sub-Reggie" % path) if isfile(join("%s/000121/sub-Reggie" %path, f))]

In [7]:
import warnings

In [52]:
dat['target_pos']

array([[  0.,   0., -70.],
       [  0.,   0., -70.],
       [100.,   0., -70.],
       ...,
       [  0.,   0., -70.],
       [-70.,   0., -70.],
       [  0.,   0., -70.]])

In [51]:
dat = load_shenoy_large("%s/000121/sub-JenkinsC/" % path + file_name, spike_threshold = 0, filter_fn='gaussian', filter_kwargs={'sigma':4}, bin_width=20, 
                        location='M1', trialize=True, interval='after_go')

../../loaders.py:644: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  raw_spike_times = np.array(nwbfile_in.units.spike_times_index)


2830 valid trials
96 valid  units
Trializing spike times


96it [00:15,  6.35it/s]


Processing spikes


100%|██████████| 2830/2830 [00:32<00:00, 88.36it/s]


Trializing Behavior


2830it [00:21, 130.05it/s]


In [40]:
for file_name in JFiles:
    with warnings.catch_warnings() as _:        
        dat = load_shenoy_large("%s/000121/sub-JenkinsC/" % path + file_name, spike_threshold = 0, filter_fn='gaussian', filter_kwargs={'sigma':4}, bin_width=20, 
                                location='M1', trialize=True)
        print("---------------------------")
        print("For file {0}".format(file_name))
        trial_lag_decodewin(file_name,dat)
        print("---------------------------")

../../loaders.py:636: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  raw_spike_times = np.array(nwbfile_in.units.spike_times_index)


2830 valid trials
96 valid  units
Trializing spike times


96it [00:15,  6.16it/s]


Processing spikes


100%|██████████| 2830/2830 [00:36<00:00, 77.92it/s]


Trializing Behavior


2830it [00:21, 133.50it/s]
/home/akumar/anaconda3/envs/dyn/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


---------------------------
For file sub-JenkinsC_ses-20150924T154125_behavior+ecephys+image.nwb
> /tmp/ipykernel_23144/3915283159.py(16)lr_decoder_crossval()
     14     X = np.squeeze(X)
     15     pdb.set_trace()
---> 16     for train_idx, test_idx in kf.split(X):
     17         # X_test = [X[idx] for idx in test_idx]
     18         # X_train = [X[idx] for idx in train_idx]



BdbQuit: 

In [62]:
for file_name in RFiles:
    with io.capture_output() as captured: ##To disable prints from loader function
        dat = load_shenoy_large("F:/dandi/000121/sub-Reggie/" + file_name, spike_threshold = 1, bin_width=10)
    print("---------------------------")
    print("For file {0}".format(file_name))
    trial_lag_decodewin(file_name,dat)
    print("---------------------------")

---------------------------
For file sub-Reggie_ses-20170118T094022_behavior+ecephys.nwb
Best pos r2: 0.0025204440260271216, with lag: 10, windows: 5
Best vel r2: 0.00023159697374340593, with lag: 5, windows: 5
Best acc r2: -5.5536157191352076e-05, with lag: 5, windows: 1
---------------------------
---------------------------
For file sub-Reggie_ses-20170119T123128_behavior+ecephys.nwb
Best pos r2: -0.0001328028297169631, with lag: 0, windows: 1
Best vel r2: -0.00011418684195493856, with lag: 10, windows: 1
Best acc r2: -8.55879881402205e-05, with lag: 10, windows: 1
---------------------------
---------------------------
For file sub-Reggie_ses-20170125T100800_behavior+ecephys.nwb
Best pos r2: 2.4098519783133376e-05, with lag: 0, windows: 1
Best vel r2: 2.6252973109736066e-05, with lag: 5, windows: 3
Best acc r2: -9.487210345915376e-05, with lag: 10, windows: 1
---------------------------
---------------------------
For file sub-Reggie_ses-20170117T104643_behavior+ecephys.nwb
Best po

In [63]:
for file_name in RFiles:
    with io.capture_output() as captured: ##To disable prints from loader function
        dat = load_shenoy_large("F:/dandi/000121/sub-Reggie/" + file_name, spike_threshold = 1, bin_width=100)
    print("---------------------------")
    print("For file {0}".format(file_name))
    trial_lag_decodewin(file_name,dat)
    print("---------------------------")

---------------------------
For file sub-Reggie_ses-20170118T094022_behavior+ecephys.nwb
Best pos r2: 0.021612839175602347, with lag: 1, windows: 3
Best vel r2: 0.0029008843985057625, with lag: 3, windows: 1
Best acc r2: 0.0006736616562762615, with lag: 3, windows: 1
---------------------------
---------------------------
For file sub-Reggie_ses-20170119T123128_behavior+ecephys.nwb
Best pos r2: -0.0020666705829109344, with lag: 0, windows: 1
Best vel r2: -0.002044313333135872, with lag: 0, windows: 1
Best acc r2: -0.0015595979350723765, with lag: 0, windows: 1
---------------------------
---------------------------
For file sub-Reggie_ses-20170125T100800_behavior+ecephys.nwb
Best pos r2: -0.00013047538006957793, with lag: 0, windows: 1
Best vel r2: 0.0011419420575776995, with lag: 1, windows: 1
Best acc r2: -0.0009386667682210331, with lag: 0, windows: 1
---------------------------
---------------------------
For file sub-Reggie_ses-20170117T104643_behavior+ecephys.nwb
Best pos r2: 0.0

In [6]:
dat.keys()

dict_keys(['target_pos', 'spike_rates', 'go_times', 'behavior', 'behavior_3D'])

In [21]:
print(dat['target_pos'].shape)
print(dat['go_times'].shape)
print(len(dat['spike_rates']))
print(len(dat['behavior']))
print(dat['behavior'][1].shape)
print(dat['spike_rates'][1].shape)

(3069, 3)
(3069,)
3069
3069
(25, 2)
(25, 40)


In [51]:
import warnings
warnings.filterwarnings("ignore")
#warnings.filterwarnings("default", category=FutureWarning)

In [64]:
def lr_decoder_crossval(X, Z, lag, decoding_window):
    kf = KFold(n_splits=4)
    lr_r2_pos= lr_r2_vel = lr_r2_acc = 0
    for train_idx, test_idx in kf.split(X):
        X_test = [X[idx] for idx in test_idx]
        X_train = [X[idx] for idx in train_idx]
        Z_test = [Z[idx] for idx in test_idx]
        Z_train = [Z[idx] for idx in train_idx]
        result = lr_decoder(X_test, X_train, Z_test, Z_train, lag, lag, decoding_window)
        lr_r2_pos += result[0]
        lr_r2_vel += result[1]
        lr_r2_acc += result[2]
    lr_r2_acc /= 4
    lr_r2_pos /= 4
    lr_r2_vel /= 4
    return lr_r2_pos, lr_r2_vel, lr_r2_acc
    

In [65]:
def trial_lag_decodewin(file_name, dat):
    X = dat['spike_rates'] 
    Z = dat['behavior']
    lag_trials = [0, 1, 3, 5, 10]
    decoding_windows = [1, 3, 5]
    max_lr_r2_pos, max_lr_r2_vel, max_lr_r2_acc = -1, -1, -1
    max_pos_lag,max_pos_decoding_window = 0,1
    max_vel_lag,max_vel_decoding_window = 0,1
    max_acc_lag,max_acc_decoding_window = 0,1
    
    
    for lag in lag_trials:
        for decoding_window in decoding_windows:
            try:
                lr_r2_pos, lr_r2_vel, lr_r2_acc = lr_decoder_crossval(X, Z, lag, decoding_window)
                if lr_r2_pos > max_lr_r2_pos:
                    max_lr_r2_pos = lr_r2_pos
                    max_pos_lag,max_pos_decoding_window = lag,decoding_window
                if lr_r2_vel > max_lr_r2_vel:
                    max_lr_r2_vel = lr_r2_vel
                    max_vel_lag,max_vel_decoding_window = lag,decoding_window
                if lr_r2_acc > max_lr_r2_acc:
                    max_lr_r2_acc = lr_r2_acc
                    max_acc_lag,max_acc_decoding_window = lag,decoding_window
            except ValueError:  #For large decoding_windows
                continue 
            #print("Lag: {0}, decoding_window: {1}".format(lag, decoding_window))
            #print("Pos R^2: {0}, Vel R^2: {1}, Acc R^2: {2}".format(lr_r2_pos, lr_r2_vel, lr_r2_acc))
    
    print("Best pos r2: {0}, with lag: {1}, windows: {2}".format(max_lr_r2_pos, max_pos_lag,max_pos_decoding_window))
    print("Best vel r2: {0}, with lag: {1}, windows: {2}".format(max_lr_r2_vel, max_vel_lag,max_vel_decoding_window))
    print("Best acc r2: {0}, with lag: {1}, windows: {2}".format(max_lr_r2_acc, max_acc_lag,max_acc_decoding_window))

In [7]:
dat['target_pos'][:,0].sum() # X-axis always 0?
dat['behavior'][3].shape
spike_rates = dat['spike_rates']
spike_rates_averaged = np.array([[np.average(spike_rates[reach_i], axis = 0)[neuron_idx] \
                                            for reach_i in range(len(spike_rates))]\
                                                for neuron_idx in range(spike_rates[0].shape[1])])
vels = [np.sum(np.gradient(dat['behavior'][i])[0] ** 2, axis = 1) for i in range(len(dat['behavior']))]
#Should we actullay be using peak vels? Or average
#‖𝑉(𝑡)‖
peak_vels = np.array([np.max(vel_in_a_reach) for vel_in_a_reach in vels])[:,np.newaxis]
# TODO: orientations = 
orientations = (np.random.rand(4753) - 0.5) * 2 * np.pi # Random for running

In [ ]:
def calc_loadings(U, d=1):
    # Sum over components
    U = np.sum(np.power(np.abs(U), 2), axis=-1)
    # Reshape and then sum over neurons
    U = np.reshape(U, (d, -1))
    loadings = np.sum(U, axis=0)
    loadings /= np.max(loadings)
    return loadings
    
def fit_DCAnPCAonBin(X):
    """Now just used to calculate PCA and DCA loading"""
    DCAmodel = DynamicalComponentsAnalysis(d=2, T=3)
    PCAmodel = PCA(n_components=2)
    DCAmodel.fit(X)
    extended = X[0]
    for transit in X[1:]:
        extended = np.vstack((extended,transit))
    PCAmodel.fit(extended)
    logged_PCA_loading = np.log(calc_loadings(PCAmodel.components_.T))
    logged_DCA_loading = np.log(calc_loadings(DCAmodel.coef_))
    return logged_PCA_loading, logged_DCA_loading

In [8]:
bins = np.arange(-np.pi,np.pi,.25 * np.pi)

#‖𝑉(𝑡)‖𝑠𝑖𝑛[𝜃(𝑡)]
vel_sin = (np.sin(orientations)[:,np.newaxis] * peak_vels)
#‖𝑉(𝑡)‖𝑐𝑜𝑠[𝜃(𝑡)]
vel_cos = (np.cos(orientations)[:,np.newaxis] * peak_vels)
binned_indices = np.digitize(orientations, bins)
binned_indices = [np.where(binned_indices == i) for i in range(1,9)]
#To record the r^2 and coeffs indexed by (bin#, neuron#)
r_squared_bin_neuron = np.zeros((len(binned_indices), spike_rates[0].shape[1]))
coefficients_bin_neuron = np.zeros((len(binned_indices), spike_rates[0].shape[1], 2))

for i in range(8):
    binned_idx = binned_indices[i][0]
    # Get the input as ‖𝑉(𝑡)‖, ‖𝑉(𝑡)‖𝑠𝑖𝑛[𝜃(𝑡)], ‖𝑉(𝑡)‖𝑐𝑜𝑠[𝜃(𝑡)]
    X = np.concatenate((peak_vels[binned_idx], vel_sin[binned_idx], vel_cos[binned_idx]), axis = 1)
    for neuron_idx in range(spike_rates[0].shape[1]):
        # Get the neuron's averaged spike rates in this bin's reaches
        Y = spike_rates_averaged[neuron_idx, binned_idx]
        reg = LinearRegression().fit(X, Y)
        #Getting r sqaured, and put it in the recording array
        r_squared_bin_neuron[i, neuron_idx] = reg.score(X, Y)
        #Do the same for coefficients for ‖𝑉(𝑡)‖𝑠𝑖𝑛[𝜃(𝑡)], ‖𝑉(𝑡)‖𝑐𝑜𝑠[𝜃(𝑡)]
        coefficients_bin_neuron[i, neuron_idx] = reg.coef_[1:]
    spike_rates_list_reach = [spike_rates[_] for _ in binned_idx]
    logged_PCA_loading, logged_DCA_loading = fit_DCAnPCAonBin(spike_rates_list_reach)

NameError: name 'fit_DCAnPCAonBin' is not defined